# Creating figures for batteries with PV plant

## Set the current path according to the operating system

In [ ]:
import sys, os

print("Operating system: %s" %sys.platform)

if sys.platform == 'win32':

    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'

elif sys.platform == 'linux':

    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'

else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)

if curr_path not in sys.path:

    sys.path.append(curr_path)

    print('Path added! \n')

os.chdir(curr_path)

print("Working directory is now %s" %curr_path)

In [ ]:
# define a function for retrieving the data from the database

#def select_data(data, years, regions, E_st_maxs, P_st_in_maxs, P_pc_out_maxs, eta_sts, oversizes, eta_invs):
#    datanow = data[([(i in years) for i in data['year'].values])&
#                   ([(i in regions) for i in data['region'].values])&
#                   ([(i in E_st_maxs) for i in data['E_st_max'].values])&
#                   ([(i in P_st_in_maxs) for i in data['P_st_in_max'].values])&
#                   ([(i in P_pc_out_maxs) for i in data['P_pc_out_max'].values])&
#                   ([(i in eta_sts) for i in np.round(data['eta_st'].values.astype(float),1)])&
#                   ([(i in oversizes) for i in np.round(data['oversize'].values.astype(float),1)])&
#                   ([(i in eta_invs) for i in data['eta_inv'].values])].set_index('year').sort_index()
#    
#    return(datanow)


def select_data(data, years, region=None, oversize=None, eta_st=None, E_st_max=None, P_st_in_max=None, P_pc_out_max=None, eta_inv=None):
    
    regionstr = ["(data['region'] == region)&", ""][region == None]
    oversizestr = ["(np.round(data['oversize'].values.astype(float),1) == np.round(oversize, 1))&", ""][oversize == None]
    eta_ststr = ["(np.round(data['eta_st'].values.astype(float),1) == np.round(eta_st, 1))&", ""][eta_st == None]
    E_st_maxstr = ["(data['E_st_max'] == E_st_max)&", ""][E_st_max == None]
    P_st_in_maxstr = ["(data['P_st_in_max'] == P_st_in_max)&", ""][P_st_in_max == None]
    P_pc_out_maxstr = ["(data['P_pc_out_max'] == P_pc_out_max)&", ""][P_pc_out_max == None]
    eta_invstr = ["(data['eta_inv'] == eta_inv)&", ""][eta_inv == None]

    totalstr = regionstr + oversizestr + eta_ststr + E_st_maxstr + P_st_in_maxstr + P_pc_out_maxstr + eta_invstr
    
    totalstr = totalstr[:-1]
    
    execstr = "datanow = data[([i in %s for i in data['year'].values])&%s].set_index('year').sort_index()" \
        %(str(years).replace(' ', ', '), totalstr)
    exec(execstr)

    return(locals()['datanow'])

## Plot the IRR vs PV plant oversize for molten salt storage

In [ ]:
from package import sql_manager as sm
import matplotlib.pyplot as plt
from projdirs import basedir
import pandas as pd
import numpy as np

plt.rcParams['text.usetex'] = True
sep = ['\\','/'][sys.platform=='linux']

excel_file = ['c:/NextCloud/Thermal Battery Research/Publications/paper_1/Data/CPE_CPP.xlsx',
              '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/Publications/paper_1/Data/CPE_CPP.xlsx'][sys.platform == 'linux']

cost_data_st = pd.read_excel(excel_file, 'Summary', header=0, usecols='A:M')

year = 2016
region = 'SA'
E_st_max = 200.
P_st_in_max = 50.
P_pc_out_max = 50.
eta_st = 0.4
eta_inv = 0.96
C_pv_cap = 1111e3
C_pv_OM = 20e3
CPE = 23.3e3
CPP = 1258e3
PV_base = 55. #MW-e(AC)
oversizes = np.arange(1.0,2.1,0.1)

lifetime = 20

exc_rate = pd.DataFrame({'year':np.arange(2010,2020,1),
                         'AUD/USD':[0.92, 1.03, 1.04, 0.97, 0.9, 0.75, 0.74, 0.77, 0.75, 0.7] }).set_index('year')

CAPEX_st = P_pc_out_max * CPP + E_st_max / eta_st * CPE
CAPEX_PV_base = PV_base * C_pv_cap
OM_PV_base = PV_base * C_pv_OM

fig = plt.figure()
ax = fig.add_subplot(111)

for region in ['SA', 'NSW']:

    IRRs = []
    
    data = sm.get_table('storage_value_PF_PVplant_summary_old.db', 'revenue').loc[region]

    for oversize in oversizes:
        datanow = data[(data['E_st_max']==E_st_max)&
                       (data['P_st_in_max']==P_st_in_max)&
                       (data['P_pc_out_max']==P_pc_out_max)&   
                       (data['eta_st']==eta_st)&
                       (data['oversize']==oversize)&
                       (data['year']>=year)].set_index('year').sort_index()

        C_in_grid_total = datanow['C_in_grid_total']
        C_out_grid_total = datanow['C_out_grid_total']

        revenue = ((C_out_grid_total - C_in_grid_total) * exc_rate[-4:]['AUD/USD']).mean()
        CAPEX = CAPEX_st + CAPEX_PV_base * oversize
        OM = OM_PV_base * oversize

        cashflows = [-CAPEX] + [revenue - OM] * lifetime

        IRRs.append(np.irr(cashflows))

    ax.plot(oversizes, IRRs, label = region)
    ax.grid(True)
    ax.axis([1.0,2.0,0.01,0.08])
    
ax.legend()
    
    
    

## Plot the IRR vs PV plant oversize for lithium ion batteries

In [ ]:
from package import sql_manager as sm
import matplotlib.pyplot as plt
from projdirs import basedir
import pandas as pd
import numpy as np
import itertools as it

plt.rcParams['text.usetex'] = True
plt.rcParams['savefig.dpi'] = 150
sep = ['\\','/'][sys.platform=='linux']

excel_file = ['c:/NextCloud/Thermal Battery Research/Publications/paper_1/Data/CPE_CPP.xlsx',
              '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/Publications/paper_1/Data/CPE_CPP.xlsx'][sys.platform == 'linux']

db = 'storage_value_PF_PVplant_summary.db'

cost_data_st = pd.read_excel(excel_file, 'Summary', header=0, usecols='A:M')

#df_cols = ['E_st_max', 'P_st_in_max',
#      'P_pc_out_max', 'oversize', 'eta_st', 'eta_inv', 
#      'year', 'P_in_grid_total', 'P_out_grid_total', 
#      'P_in_plant_total', 'P_out_pass_total', 
#      'P_inv_in_total', 'P_st_in_plant_cur_total',
#      'P_st_in_plant_vol_total', 'C_in_grid_total', 
#      'C_out_grid_total', 'C_out_pass_total',
#      'C_inv_out_total', 'C_st_in_plant_cur_total',
#      'C_st_in_plant_vol_total', 'region']

#years, regions, oversizes, eta_sts, E_st_maxs, P_st_in_maxs, 
#                 P_pc_out_maxs, eta_invs, C_pv_caps, C_pv_OMs, CPEs, CPPs, 
#                 PV_base = 55., lifetime = 20

#def extract_data(data, data_arrays, years, regions, oversizes, eta_sts, E_st_maxs, P_st_in_maxs, 
#                 P_pc_out_maxs, eta_invs):
#    "All arguments must be list or numpy array types"

#    argvals = it.product(years, regions, oversizes, eta_sts, E_st_maxs, P_st_in_maxs,
#                         P_pc_out_maxs, eta_invs)
    
#    argsdict = np.array([dict({'year':i[0], 'region':i[1], 'oversize':i[2], 'eta_st':i[3],
#                               'E_st_max':i[4], 'P_st_in_max':i[5], 'eta_inv':i[6]}) for i in argvals])

eta_st = 0.9

fig1 = plt.figure()
ax1 = fig1.add_subplot(111)

fig2 = plt.figure(figsize = (200/25.4, 80/25.4))
fig2.suptitle('Storage cost, eta\_st = %.1f' %eta_st)
ax21 = fig2.add_subplot(121)
ax21.set_title('NSW')
ax21.grid(True, zorder = 1)
ax22 = fig2.add_subplot(122)
ax22.set_yticklabels([])
ax22.set_title('SA')
ax22.grid(True, zorder = 1)

fig3 = plt.figure(figsize = (200/25.4, 80/25.4))
fig3.suptitle('Generation revenue, eta\_st = %.1f' %eta_st)
ax31 = fig3.add_subplot(121)
ax31.set_title('NSW')
ax31.grid(True, zorder = 1)
ax32 = fig3.add_subplot(122)
ax32.set_yticklabels([])
ax32.set_title('SA')
ax32.grid(True, zorder = 1)

fig4 = plt.figure(figsize = (200/25.4, 80/25.4))
fig4.suptitle('Stored electrical energy, eta\_st = %.1f' %eta_st)
ax41 = fig4.add_subplot(121)
ax41.set_title('NSW')
ax41.grid(True, zorder = 1)
ax42 = fig4.add_subplot(122)
ax42.set_yticklabels([])
ax42.set_title('SA')
ax42.grid(True, zorder = 1)

fig5 = plt.figure(figsize = (200/25.4, 80/25.4))
fig5.suptitle('Generated electrical energy, eta\_st = %.1f' %eta_st)
ax51 = fig5.add_subplot(121)
ax51.set_title('NSW')
ax51.grid(True, zorder = 1)
ax52 = fig5.add_subplot(122)
ax52.set_yticklabels([])
ax52.set_title('SA')
ax52.grid(True, zorder = 1)

fig6 = plt.figure(figsize = (200/25.4, 80/25.4))
fig6.suptitle('Storage value-add factor, eta\_st = %.1f' %eta_st)
ax61 = fig6.add_subplot(121)
ax61.set_title('NSW')
ax61.grid(True, zorder = 1)
ax62 = fig6.add_subplot(122)
ax62.set_yticklabels([])
ax62.set_title('SA')
ax62.grid(True, zorder = 1)

fig7 = plt.figure(figsize = (200/25.4, 80/25.4))
fig7.suptitle('Revenue per MWh, eta\_st = %.1f' %eta_st)
ax71 = fig7.add_subplot(121)
ax71.set_title('NSW')
ax71.grid(True, zorder = 1)
ax72 = fig7.add_subplot(122)
ax72.set_yticklabels([])
ax72.set_title('SA')
ax72.grid(True, zorder = 1)



year = 2016
region = 'SA'
E_st_max = 200.
P_st_in_max = 50.
P_pc_out_max = 50.
eta_inv = 0.96
C_pv_cap = 1111e3
C_pv_OM = 20e3
CPE = 66.67e3
CPP = 208.33e3
PV_base = 55. #MW-e(AC)
oversizes = np.arange(1.0,2.1,0.1)

lifetime = 20

    
exc_rate = pd.DataFrame({'year':np.arange(2010,2020,1),
                     'AUD/USD':[0.92, 1.03, 1.04, 0.97, 0.9, 0.75, 0.74, 
                                0.77, 0.75, 0.7] }).set_index('year')

CAPEX_st = P_pc_out_max * CPP + E_st_max / eta_st * CPE
CAPEX_PV_base = PV_base * C_pv_cap
OM_PV_base = PV_base * C_pv_OM

years = np.arange(2016, 2020)

table = sm.list_tables(db)[0]
cols = [col[0] for col in sm.list_columns(db,table)]
data = sm.get_data(cols,table,db)

C_pur = []
C_gen = []
C_pass = []
C_inv = []
C_cur = []
C_vol = []
C_st_out = []
C_st_tot = []
C_st_val = []

P_pur = []
P_gen = []
P_pass = []
P_inv = []
P_cur = []
P_vol = []
P_st_out = []


IRRs = []

for region in ['SA', 'NSW']:

    C_pur += [[]]
    C_gen += [[]]
    C_pass += [[]]                               
    C_inv += [[]]
    C_cur += [[]]
    C_vol += [[]]
    C_st_out += [[]]
    C_st_tot += [[]]
    C_st_val += [[]]
    
    P_pur += [[]]
    P_gen += [[]]
    P_pass += [[]]                               
    P_inv += [[]]
    P_cur += [[]]
    P_vol += [[]]
    P_st_out += [[]]
    
    IRRs += [[]]
    
    for oversize in oversizes:
        datanow = select_data(data, year, region, oversize, eta_st, E_st_max, P_st_in_max, P_pc_out_max, eta_inv)

        C_pur[-1] += [(datanow['C_in_grid_total'] * exc_rate.loc[years]['AUD/USD']).mean()]
        C_gen[-1] += [(datanow['C_out_grid_total'] * exc_rate.loc[years]['AUD/USD']).mean()]
        C_pass[-1] += [(datanow['C_out_pass_total'] * exc_rate.loc[years]['AUD/USD']).mean()]
        C_inv[-1] += [(datanow['C_inv_out_total'] * exc_rate.loc[years]['AUD/USD']).mean()]
        C_cur[-1] += [(datanow['C_st_in_plant_cur_total'] * exc_rate.loc[years]['AUD/USD']).mean()]
        C_vol[-1] += [(datanow['C_st_in_plant_vol_total'] * exc_rate.loc[years]['AUD/USD']).mean()]

        C_st_out[-1] = np.array(C_gen[-1]) - np.array(C_inv[-1])
        C_st_tot[-1] = np.array(C_pur[-1]) + np.array(C_cur[-1]) + np.array(C_vol[-1])
        C_st_val[-1] = C_st_out[-1] / C_st_tot[-1]
        
        P_pur[-1] += [(datanow['P_in_grid_total'])]
        P_gen[-1] += [(datanow['P_out_grid_total'])]
        P_pass[-1] += [(datanow['P_out_pass_total'])]
        P_inv[-1] += [(datanow['P_inv_in_total'] * eta_inv)] 
        P_cur[-1] += [(datanow['P_st_in_plant_cur_total'])] 
        P_vol[-1] += [(datanow['P_st_in_plant_vol_total'])] 

        P_st_out[-1] = np.array(P_gen[-1]) - np.array(P_inv[-1])
        
        revenue = C_gen[-1][-1] - C_pur[-1][-1]
        CAPEX = CAPEX_st + CAPEX_PV_base * oversize
        OM = OM_PV_base * oversize

        cashflows = [-CAPEX] + [revenue - OM] * lifetime

        IRRs[-1] += [np.irr(cashflows)]

        if region == 'NSW':
            ax21.bar(oversize, C_pur[-1][-1], width = 0.05, label = 'purchased', color = 'b', zorder = 2)
            ax21.bar(oversize, C_cur[-1][-1], width = 0.05, bottom = C_pur[-1][-1], label = 'curtailed', color = 'orange', zorder = 2)
            ax21.bar(oversize, C_vol[-1][-1], width = 0.05, bottom = C_pur[-1][-1] + C_cur[-1][-1], label = 'voluntary', color = 'g', zorder = 2)
            #ax2.bar(oversize, C_pass[-1][-1], width = 0.05, bottom = C_pur[-1][-1] + C_cur[-1][-1] + C_vol[-1][-1], label = 'bypass', color = 'r')
            ax31.bar(oversize, C_inv[-1][-1], width = 0.05, label = 'PV out', color = 'b', zorder = 2)
            ax31.bar(oversize, C_st_out[-1][-1], width = 0.05, bottom = C_inv[-1][-1], label = 'Storage out', color = 'orange', zorder = 2)

            ax41.bar(oversize, P_pur[-1][-1], width = 0.05, label = 'purchased', color = 'b', zorder = 2)
            ax41.bar(oversize, P_cur[-1][-1], width = 0.05, bottom = P_pur[-1][-1], label = 'curtailed', color = 'orange', zorder = 2)
            ax41.bar(oversize, P_vol[-1][-1], width = 0.05, bottom = P_pur[-1][-1] + P_cur[-1][-1], label = 'voluntary', color = 'g', zorder = 2)

            ax51.bar(oversize, P_inv[-1][-1], width = 0.05, label = 'PV out', color = 'b', zorder = 2)
            ax51.bar(oversize, P_st_out[-1][-1], width = 0.05, bottom = P_inv[-1][-1], label = 'Storage out', color = 'orange', zorder = 2)
            
            ax61.bar(oversize, C_st_val[-1][-1], width = 0.05, color = 'b', zorder = 2)
            
            ax71.bar(oversize-0.02, C_inv[-1][-1]/P_inv[-1][-1], width = 0.04, color = 'b', label = 'PV out', zorder = 2)
            ax71.bar(oversize+0.02, C_st_out[-1][-1]/P_st_out[-1][-1], width = 0.04, color = 'orange', label = 'storage out', zorder = 2)
            
            
        if region == 'SA':
            ax22.bar(oversize, C_pur[-1][-1], width = 0.05, label = 'purchased', color = 'b', zorder = 2)
            ax22.bar(oversize, C_cur[-1][-1], width = 0.05, bottom = C_pur[-1][-1], label = 'curtailed', color = 'orange', zorder = 2)
            ax22.bar(oversize, C_vol[-1][-1], width = 0.05, bottom = C_pur[-1][-1] + C_cur[-1][-1], label = 'voluntary', color = 'g', zorder = 2)
            
            ax32.bar(oversize, C_inv[-1][-1], width = 0.05, label = 'PV out', color = 'b', zorder = 2)
            ax32.bar(oversize, C_st_out[-1][-1], width = 0.05, bottom = C_inv[-1][-1], label = 'Storage out', color = 'orange', zorder = 2)

            ax42.bar(oversize, P_pur[-1][-1], width = 0.05, label = 'purchased', color = 'b', zorder = 2)
            ax42.bar(oversize, P_cur[-1][-1], width = 0.05, bottom = P_pur[-1][-1], label = 'curtailed', color = 'orange', zorder = 2)
            ax42.bar(oversize, P_vol[-1][-1], width = 0.05, bottom = P_pur[-1][-1] + P_cur[-1][-1], label = 'voluntary', color = 'g', zorder = 2)

            ax52.bar(oversize, P_inv[-1][-1], width = 0.05, label = 'PV out', color = 'b', zorder = 2)
            ax52.bar(oversize, P_st_out[-1][-1], width = 0.05, bottom = P_inv[-1][-1], label = 'Storage out', color = 'orange', zorder = 2)

            ax62.bar(oversize, C_st_val[-1][-1], width = 0.05, color = 'b', zorder = 2)
            
            ax72.bar(oversize-0.02, C_inv[-1][-1]/P_inv[-1][-1], width = 0.04, color = 'b', label = 'PV out', zorder = 2)
            ax72.bar(oversize+0.02, C_st_out[-1][-1]/P_st_out[-1][-1], width = 0.04, color = 'orange', label = 'storage out', zorder = 2)
            
            
    ax1.plot(oversizes, IRRs[-1], label = region)

ax1.grid(True)

ax1.set_xlim([1.0,2.0])

ax1.legend()

ax2_handles, ax2_labels = ax21.get_legend_handles_labels()
ax2_label = dict(zip(ax2_labels, ax2_handles))

ax3_handles, ax3_labels = ax31.get_legend_handles_labels()
ax3_label = dict(zip(ax3_labels, ax3_handles))

ax4_handles, ax4_labels = ax41.get_legend_handles_labels()
ax4_label = dict(zip(ax4_labels, ax4_handles))

ax5_handles, ax5_labels = ax51.get_legend_handles_labels()
ax5_label = dict(zip(ax5_labels, ax5_handles))

ax7_handles, ax7_labels = ax71.get_legend_handles_labels()
ax7_label = dict(zip(ax7_labels, ax7_handles))

ax21.legend(ax2_label.values(), ax2_label.keys())
ax21.set_ylim([0,1e7])
ax22.legend(ax2_label.values(), ax2_label.keys())
ax22.set_ylim([0,1e7])

ax31.legend(ax3_label.values(), ax3_label.keys())
ax31.set_ylim([0,4e7])
ax32.legend(ax3_label.values(), ax3_label.keys())
ax32.set_ylim([0,4e7])

ax41.legend(ax4_label.values(), ax4_label.keys())
ax41.set_ylim([0,2e5])
ax42.legend(ax4_label.values(), ax4_label.keys())
ax42.set_ylim([0,2e5])

ax51.legend(ax5_label.values(), ax5_label.keys())
ax51.set_ylim([0,2.5e5])
ax52.legend(ax5_label.values(), ax5_label.keys())
ax52.set_ylim([0,2.5e5])

ax61.set_ylim([0,3])

ax62.set_ylim([0,3])
ax62.grid(True)

ax71.legend(ax7_label.values(), ax7_label.keys())
ax71.set_ylim([0,200])
ax72.legend(ax7_label.values(), ax7_label.keys())
ax72.set_ylim([0,200])

savedir = basedir + 'Publications/paper_2/Figures/'

fig2.savefig(savedir + 'storage_cost_%i.png' %(eta_st*100))
fig3.savefig(savedir + 'generation_revenue_%i.png' %(eta_st*100))
fig4.savefig(savedir + 'stored_electrical_energy_%i.png' %(eta_st*100))
fig5.savefig(savedir + 'generated_electrical_energy_%i.png' %(eta_st*100))
fig6.savefig(savedir + 'storage_value-add_factor_%i.png' %(eta_st*100))
fig7.savefig(savedir + 'revenue_per_MWh_%i.png' %(eta_st*100))


In [ ]:
from package import sql_manager as sm
import matplotlib.pyplot as plt
from projdirs import basedir
import pandas as pd
import numpy as np
import itertools as it

plt.rcParams['text.usetex'] = True
plt.rcParams['savefig.dpi'] = 150
sep = ['\\','/'][sys.platform=='linux']

excel_file = ['c:/NextCloud/Thermal Battery Research/Publications/paper_1/Data/CPE_CPP.xlsx',
              '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/Publications/paper_1/Data/CPE_CPP.xlsx'][sys.platform == 'linux']

db = 'storage_value_PF_PVplant_summary.db'

cost_data_st = pd.read_excel(excel_file, 'Summary', header=0, usecols='A:M')

eta_st = 0.4

fig1 = plt.figure()
ax1 = fig1.add_subplot(111)

fig3 = plt.figure(figsize = (200/25.4, 80/25.4))
fig3.suptitle('Generation revenue, eta\_st = %.1f' %eta_st)
ax31 = fig3.add_subplot(121)
ax31.set_title('NSW')
ax31.grid(True, zorder = 1)
ax32 = fig3.add_subplot(122)
ax32.set_yticklabels([])
ax32.set_title('SA')
ax32.grid(True, zorder = 1)

fig5 = plt.figure(figsize = (200/25.4, 80/25.4))
fig5.suptitle('Generated electrical energy, eta\_st = %.1f' %eta_st)
ax51 = fig5.add_subplot(121)
ax51.set_title('NSW')
ax51.grid(True, zorder = 1)
ax52 = fig5.add_subplot(122)
ax52.set_yticklabels([])
ax52.set_title('SA')
ax52.grid(True, zorder = 1)

fig7 = plt.figure(figsize = (200/25.4, 80/25.4))
fig7.suptitle('Revenue per MWh, eta\_st = %.1f' %eta_st)
ax71 = fig7.add_subplot(121)
ax71.set_title('NSW')
ax71.grid(True, zorder = 1)
ax72 = fig7.add_subplot(122)
ax72.set_yticklabels([])
ax72.set_title('SA')
ax72.grid(True, zorder = 1)

year = 2016
E_st_max = 0.
P_st_in_max = 0.
P_pc_out_max = 1e-8
eta_inv = 0.96
C_pv_cap = 1111e3
C_pv_OM = 20e3
CPE = 66.67e3
CPP = 208.33e3
PV_base = 50. #MW-e(AC)
oversizes = np.arange(1.0,2.1,0.1)

lifetime = 20

exc_rate = pd.DataFrame({'year':np.arange(2010,2020,1),
                     'AUD/USD':[0.92, 1.03, 1.04, 0.97, 0.9, 0.75, 0.74, 
                                0.77, 0.75, 0.7] }).set_index('year')

CAPEX_PV_base = PV_base * C_pv_cap
OM_PV_base = PV_base * C_pv_OM

years = np.arange(2016, 2020)

table = sm.list_tables(db)[0]
print(table)
cols = [col[0] for col in sm.list_columns(db,table)]
data = sm.get_data(cols,table,db)

C_gen = []
C_inv = []
P_gen = []
P_inv = []
P_pass = []
P_cur = []
P_vol = []
IRRs = []

for region in ['SA','NSW']:

    C_gen += [[]]
    C_inv += [[]]
    P_gen += [[]]
    P_inv += [[]]
    P_pass += [[]]
    P_cur += [[]]
    P_vol += [[]]
    IRRs += [[]]
    
    for oversize in oversizes:
        
        datanow = select_data(data, year, region, oversize, eta_st, E_st_max, P_st_in_max, P_pc_out_max, eta_inv)

        C_gen[-1] += [(datanow['C_out_grid_total'] * exc_rate.loc[years]['AUD/USD']).mean()]
        P_gen[-1] += [(datanow['P_out_grid_total'] * exc_rate.loc[years]['AUD/USD']).mean()]
        C_inv[-1] += [(datanow['C_inv_out_total'] * exc_rate.loc[years]['AUD/USD']).mean()]
        P_inv[-1] += [(datanow['P_inv_in_total'] * eta_inv)]
        P_pass[-1] += [(datanow['P_out_pass_total'])]
        P_cur[-1] += [(datanow['P_st_in_plant_cur_total'])]
        P_vol[-1] += [(datanow['P_st_in_plant_vol_total'])]
        
        revenue = C_gen[-1][-1]
        CAPEX = CAPEX_PV_base * oversize
        OM = OM_PV_base * oversize

        cashflows = [-CAPEX] + [revenue - OM] * lifetime

        IRRs[-1] += [np.irr(cashflows)]

        if region == 'NSW':
            ax31.bar(oversize, C_inv[-1][-1], width = 0.05, label = 'PV out', color = 'b', zorder = 2)
            ax51.bar(oversize, P_inv[-1][-1], width = 0.05, label = 'PV out', color = 'b', zorder = 2)
            ax71.bar(oversize-0.02, C_inv[-1][-1]/P_inv[-1][-1], width = 0.04, color = 'b', label = 'PV out', zorder = 2)
            
        if region == 'SA':
            ax32.bar(oversize, C_inv[-1][-1], width = 0.05, label = 'PV out', color = 'b', zorder = 2)
            ax52.bar(oversize, P_inv[-1][-1], width = 0.05, label = 'PV out', color = 'b', zorder = 2)
            ax72.bar(oversize-0.02, C_inv[-1][-1]/P_inv[-1][-1], width = 0.04, color = 'b', label = 'PV out', zorder = 2)
        
    ax1.plot(oversizes, IRRs[-1], label = region)


    
ax1.grid(True)

ax1.set_xlim([1.0,2.0])

ax1.legend()

ax3_handles, ax3_labels = ax31.get_legend_handles_labels()
ax3_label = dict(zip(ax3_labels, ax3_handles))

ax5_handles, ax5_labels = ax51.get_legend_handles_labels()
ax5_label = dict(zip(ax5_labels, ax5_handles))

ax7_handles, ax7_labels = ax71.get_legend_handles_labels()
ax7_label = dict(zip(ax7_labels, ax7_handles))

ax31.legend(ax3_label.values(), ax3_label.keys())
ax31.set_ylim([0,1.5e7])
ax32.legend(ax3_label.values(), ax3_label.keys())
ax32.set_ylim([0,1.5e7])

ax51.legend(ax5_label.values(), ax5_label.keys())
ax51.set_ylim([0,1.5e5])
ax52.legend(ax5_label.values(), ax5_label.keys())
ax52.set_ylim([0,1.5e5])

ax71.legend(ax7_label.values(), ax7_label.keys())
ax71.set_ylim([0,150])
ax72.legend(ax7_label.values(), ax7_label.keys())
ax72.set_ylim([0,150])

savedir = basedir + 'Publications/paper_2/Figures/'

fig3.savefig(savedir + 'generation_revenue_%i_PVonly.png' %(eta_st*100))
fig5.savefig(savedir + 'generated_electrical_energy_%i_PVonly.png' %(eta_st*100))
fig7.savefig(savedir + 'revenue_per_MWh_%i_PVonly.png' %(eta_st*100))


## No PV case

In [ ]:
from package import sql_manager as sm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from projdirs import basedir
import pandas as pd
import numpy as np
import itertools as it

plt.rcParams['text.usetex'] = True
plt.rcParams['savefig.dpi'] = 150
sep = ['\\','/'][sys.platform=='linux']

excel_file = ['c:/NextCloud/Thermal Battery Research/Publications/paper_1/Data/CPE_CPP.xlsx',
              '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/Publications/paper_1/Data/CPE_CPP.xlsx'][sys.platform == 'linux']

db = 'storage_value_PF_PVplant_summary.db'

cost_data_st = pd.read_excel(excel_file, 'Summary', header=0, usecols='A:M')

SH = 4
eta_st = 0.4
oversize = 0
P_pc_out_maxs_range = np.arange(10,101,10)
regions_list = ['NSW', 'SA']
regions, P_pc_out_maxs = np.meshgrid(regions_list, 
                                     P_pc_out_maxs_range, 
                                     indexing = 'ij')

P_st_in_maxs = np.round(P_pc_out_maxs / eta_st / 5) * 5
E_st_maxs = np.round(P_pc_out_maxs * SH / 5) * 5

C_pur_nopv = np.zeros(regions.shape)
C_gen_nopv = np.zeros(regions.shape)
P_pur_nopv = np.zeros(regions.shape)
P_gen_nopv = np.zeros(regions.shape)
IRRs_nopv = np.zeros(regions.shape)
C_gen_debug = np.empty(regions.shape, dtype = object)

year = 2016
eta_inv = 0.96
lifetime = 20

exc_rate = pd.DataFrame({'year':np.arange(2010,2020,1),
                     'AUD/USD':[0.92, 1.03, 1.04, 0.97, 0.9, 0.75, 0.74, 
                                0.77, 0.75, 0.7] }).set_index('year')

years = np.arange(2016, 2020)
table = sm.list_tables(db)[0]
print(table)
cols = [col[0] for col in sm.list_columns(db,table)]
data = sm.get_data(cols,table,db)

for x, y in [i for i in it.product(range(len(regions_list)),
                                   range(len(P_pc_out_maxs_range)))]:
       
    datanow = select_data(data, year, regions[x,y], oversize, eta_st, 
                          P_pc_out_max = P_pc_out_maxs[x,y], E_st_max = E_st_maxs[x,y],
                          P_st_in_max = P_st_in_maxs[x,y])

    C_pur_nopv[x,y] = (datanow['C_in_grid_total'] * exc_rate.loc[years]['AUD/USD']).mean()
    C_gen_nopv[x,y] = (datanow['C_out_grid_total'] * exc_rate.loc[years]['AUD/USD']).mean()
    P_pur_nopv[x,y] = datanow['P_in_grid_total']
    P_gen_nopv[x,y] = datanow['P_out_grid_total']
    
    print(regions[x,y], P_pc_out_maxs[x,y])
    print(datanow['C_out_grid_total'])
    
    

#CPE = 66.67e3
#CPP = 208.33e3
CPE = 23.3e3
CPP = 1258e3
CAPEX_st = P_pc_out_maxs * CPP + E_st_maxs / eta_st * CPE
CAPEX_nopv = CAPEX_st

C_st_out_nopv = C_gen_nopv
C_st_tot_nopv = C_pur_nopv
C_st_val_nopv = C_st_out_nopv / C_st_tot_nopv
P_st_out_nopv = P_gen_nopv

revenue_nopv = C_gen_nopv - C_pur_nopv
CAPEX_nopv_ravel = np.ravel(CAPEX_nopv)
rev_nopv_ravel = np.ravel(revenue_nopv)
#cashflows = [-CAPEX] + [revenue - OM] * lifetime

#IRRs = np.array([np.irr([-CAPEX] + [rev_ravel[i] - OM_ravel[i]] * lifetime) for i in range(len(rev_ravel))]).reshape(OM.shape)

IRRs_nopv = np.array([np.irr([-CAPEX_nopv_ravel[i]] + [rev_nopv_ravel[i]] * lifetime) \
                        for i in range(len(rev_nopv_ravel))]).reshape(IRRs_nopv.shape)

fig1 = plt.figure(figsize = (240/25.4, 100/25.4))
ax11 = fig1.add_subplot(131)
ax11.set_title('IRR')
ax11.grid(True, zorder = 1)
ax11.set_xlim([0,100])

ax11.plot(P_pc_out_maxs[0], IRRs_nopv[0])
ax11.plot(P_pc_out_maxs[1], IRRs_nopv[1])
ax11.set_ylim([np.amin(IRRs_nopv) * (1.0 - 0.1*np.sign(np.amin(IRRs_nopv))), 1.1*np.amax([IRRs_nopv])])

ax12 = fig1.add_subplot(132)
ax12.set_title('revenue')
ax12.grid(True, zorder = 1)
ax12.set_xlim([0,100])

ax12.plot(P_pc_out_maxs[0], C_gen_nopv[0])
ax12.plot(P_pc_out_maxs[1], C_gen_nopv[1])
ax12.set_ylim([np.amin(C_gen_nopv) * (1.0 - 0.1*np.sign(np.amin(C_gen_nopv))), 
               1.1*np.amax([C_gen_nopv])])

In [ ]:
from package import sql_manager as sm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from projdirs import basedir
import pandas as pd
import numpy as np
import itertools as it

plt.rcParams['text.usetex'] = True
plt.rcParams['savefig.dpi'] = 150
sep = ['\\','/'][sys.platform=='linux']

excel_file = ['c:/NextCloud/Thermal Battery Research/Publications/paper_1/Data/CPE_CPP.xlsx',
              '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/Publications/paper_1/Data/CPE_CPP.xlsx'][sys.platform == 'linux']

db = 'storage_value_PF_PVplant_summary.db'

cost_data_st = pd.read_excel(excel_file, 'Summary', header=0, usecols='A:M')

eta_st = 0.4

region_list = ['NSW', 'SA']
plot_list = ['IRR ','NoStor ','Stor IRR+ ', 'NoPV ', 'PV IRR+ ']

fig1 = plt.figure(figsize = (160/25.4, 1000/25.4))
ax1 = np.empty(len(plot_list), dtype = object)

for i in range(len(plot_list)):
    ax1[i] = fig1.add_subplot(5,1,i+1, projection = '3d')
    ax1[i].set_title(plot_list[i])
    ax1[i].grid(True, color = 'k', linestyle = ':', zorder = 1)
    ax1[i].set_xlim([1,2])
    ax1[i].set_ylim([0,100])


fig8 = plt.figure(figsize = (360/25.4, 1000/25.4))
ax8_titles = [i for i in it.product(plot_list, region_list)]
ax8 = np.empty(len(ax8_titles), dtype = object)

for i in range(len(ax8_titles)):

    ax8[i] = fig8.add_subplot(5,2,i+1)
    ax8[i].set_title(ax8_titles[i][0]+ax8_titles[i][1])
    ax8[i].grid(color = 'k', linestyle = ':')
    

fig2 = plt.figure(figsize = (200/25.4, 80/25.4))
fig2.suptitle('Storage cost, eta\_st = %.1f' %eta_st)
ax21 = fig2.add_subplot(121, projection = '3d')
ax21.set_title('NSW')
ax21.grid(True, zorder = 1)
ax22 = fig2.add_subplot(122, projection = '3d')
ax22.set_title('SA')
ax22.grid(True, zorder = 1)

fig3 = plt.figure(figsize = (200/25.4, 80/25.4))
fig3.suptitle('Generation revenue, eta\_st = %.1f' %eta_st)
ax31 = fig3.add_subplot(121, projection = '3d')
ax31.set_title('NSW')
ax31.grid(True, zorder = 1)
ax32 = fig3.add_subplot(122, projection = '3d')
ax32.set_title('SA')
ax32.grid(True, zorder = 1)

fig4 = plt.figure(figsize = (200/25.4, 80/25.4))
fig4.suptitle('Stored electrical energy, eta\_st = %.1f' %eta_st)
ax41 = fig4.add_subplot(121, projection = '3d')
ax41.set_title('NSW')
ax41.grid(True, zorder = 1)
ax42 = fig4.add_subplot(122, projection = '3d')
ax42.set_title('SA')
ax42.grid(True, zorder = 1)

fig5 = plt.figure(figsize = (200/25.4, 80/25.4))
fig5.suptitle('Generated electrical energy, eta\_st = %.1f' %eta_st)
ax51 = fig5.add_subplot(121, projection = '3d')
ax51.set_title('NSW')
ax51.grid(True, zorder = 1)
ax52 = fig5.add_subplot(122, projection = '3d')
ax52.set_title('SA')
ax52.grid(True, zorder = 1)

fig6 = plt.figure(figsize = (200/25.4, 80/25.4))
fig6.suptitle('Storage value-add factor, eta\_st = %.1f' %eta_st)
ax61 = fig6.add_subplot(121, projection = '3d')
ax61.set_title('NSW')
ax61.grid(True, zorder = 1)
ax61.set_xlim([1,2])
ax61.set_ylim([0,100])
ax62 = fig6.add_subplot(122, projection = '3d')
ax62.set_title('SA')
ax62.grid(True, zorder = 1)
ax62.set_xlim([1,2])
ax62.set_ylim([0,100])

fig7 = plt.figure(figsize = (200/25.4, 80/25.4))
fig7.suptitle('Revenue per MWh, eta\_st = %.1f' %eta_st)
ax71 = fig7.add_subplot(121, projection = '3d')
ax71.set_title('NSW')
ax71.grid(True, zorder = 1)
ax71.set_xlim([1,2])
ax71.set_ylim([0,100])
ax72 = fig7.add_subplot(122, projection = '3d')
ax72.set_title('SA')
ax72.grid(True, zorder = 1)
ax72.set_xlim([1,2])
ax72.set_ylim([0,100])

SH = 4
oversizes_range = np.arange(1.0, 2.1, 0.1)
P_pc_out_maxs_range = np.arange(10,101,10)

regions, oversizes, P_pc_out_maxs = np.meshgrid(regions_list, 
                                                oversizes_range, 
                                                P_pc_out_maxs_range, 
                                                indexing = 'ij')
E_st_maxs = np.round(P_pc_out_maxs * SH / 5) * 5
P_st_in_maxs = np.round(P_pc_out_maxs / eta_st / 5) * 5

year = 2016
eta_inv = 0.96
C_pv_cap = 1111e3
C_pv_OM = 20e3
CPE = 25e3
CPP = 1350e3

#CPE = 66.67e3
#CPP = 208.33e3
PV_base = 50. #MW-e(AC)

lifetime = 20

exc_rate = pd.DataFrame({'year':np.arange(2010,2020,1),
                      'AUD/USD':[0.92, 1.03, 1.04, 0.97, 
                                 0.9, 0.75, 0.74, 0.77, 
                                 0.75, 0.7] }).set_index('year')

OM = PV_base * C_pv_OM * oversizes
CAPEX_pv = PV_base * C_pv_cap * oversizes
CAPEX_st = P_pc_out_maxs * CPP + E_st_maxs / eta_st * CPE
CAPEX = CAPEX_pv + CAPEX_st
CAPEX_nostor = CAPEX - CAPEX_st
CAPEX_nopv = CAPEX - CAPEX_pv

years = np.arange(2016, 2020)

table = sm.list_tables(db)[0]
print(table)
cols = [col[0] for col in sm.list_columns(db,table)]
data = sm.get_data(cols,table,db)

C_pur = np.zeros(oversizes.shape)
C_gen = np.zeros(oversizes.shape)
C_pass = np.zeros(oversizes.shape)
C_inv = np.zeros(oversizes.shape)
C_cur = np.zeros(oversizes.shape)
C_vol = np.zeros(oversizes.shape)
P_pur = np.zeros(oversizes.shape)
P_gen = np.zeros(oversizes.shape)
P_pass = np.zeros(oversizes.shape)
P_inv = np.zeros(oversizes.shape)
P_cur = np.zeros(oversizes.shape)
P_vol = np.zeros(oversizes.shape)

C_pur_nostor = np.zeros(oversizes.shape)
C_gen_nostor = np.zeros(oversizes.shape)
C_pass_nostor = np.zeros(oversizes.shape)
C_inv_nostor = np.zeros(oversizes.shape)
P_pur_nostor = np.zeros(oversizes.shape)
P_gen_nostor = np.zeros(oversizes.shape)
P_pass_nostor = np.zeros(oversizes.shape)
P_inv_nostor = np.zeros(oversizes.shape)

C_pur_nopv = np.zeros(oversizes.shape)
C_gen_nopv = np.zeros(oversizes.shape)
P_pur_nopv = np.zeros(oversizes.shape)
P_gen_nopv = np.zeros(oversizes.shape)

for x, y, z in [i for i in it.product(range(oversizes.shape[0]),
                                      range(oversizes.shape[1]), 
                                      range(oversizes.shape[2]))]:

    datanow = select_data(data, years, regions[x,y,z], oversizes[x,y,z], eta_st, 
                          P_pc_out_max = P_pc_out_maxs[x,y,z], E_st_max = E_st_maxs[x,y,z],
                          P_st_in_max = P_st_in_maxs[x,y,z])

#    data, year, region=None, oversize=None, eta_st=None, 
#                E_st_max=None, P_st_in_max=None, P_pc_out_max=None, eta_inv=None
    
    C_pur[x,y,z] = (datanow['C_in_grid_total'] * exc_rate.loc[years]['AUD/USD']).mean()
    C_gen[x,y,z] = (datanow['C_out_grid_total'] * exc_rate.loc[years]['AUD/USD']).mean()
    C_pass[x,y,z] = (datanow['C_out_pass_total'] * exc_rate.loc[years]['AUD/USD']).mean()
    C_inv[x,y,z] = (datanow['C_inv_out_total'] * exc_rate.loc[years]['AUD/USD']).mean()
    C_cur[x,y,z] = (datanow['C_st_in_plant_cur_total'] * exc_rate.loc[years]['AUD/USD']).mean()
    C_vol[x,y,z] = (datanow['C_st_in_plant_vol_total'] * exc_rate.loc[years]['AUD/USD']).mean()
    P_pur[x,y,z] = datanow['P_in_grid_total'].mean()
    P_gen[x,y,z] = datanow['P_out_grid_total'].mean()
    P_pass[x,y,z] = datanow['P_out_pass_total'].mean()
    P_inv[x,y,z] = datanow['P_inv_in_total'].mean() * eta_inv
    P_cur[x,y,z] = datanow['P_st_in_plant_cur_total'].mean()
    P_vol[x,y,z] = datanow['P_st_in_plant_vol_total'].mean()

for x, y in [i for i in it.product(range(len(regions_list)),
                                   range(len(oversizes_range)))]:
    
    P_pc_out_max = 1e-8
    
    datanow = select_data(data, years, regions[x,y,0], oversizes[x,y,0], 
                          P_pc_out_max = P_pc_out_max)
    
#    print(datanow['C_out_grid_total'])
#    data, year, region=None, oversize=None, eta_st=None, 
#                E_st_max=None, P_st_in_max=None, P_pc_out_max=None, eta_inv=None
    
    C_pur_nostor[x,y,:] = (datanow['C_in_grid_total'] * exc_rate.loc[years]['AUD/USD']).mean()
    C_gen_nostor[x,y,:] = (datanow['C_out_grid_total'] * exc_rate.loc[years]['AUD/USD']).mean()
    C_pass_nostor[x,y,:] = (datanow['C_out_pass_total'] * exc_rate.loc[years]['AUD/USD']).mean()
    C_inv_nostor[x,y,:] = (datanow['C_inv_out_total'] * exc_rate.loc[years]['AUD/USD']).mean()
    P_pur_nostor[x,y,:] = datanow['P_in_grid_total'].mean()
    P_gen_nostor[x,y,:] = datanow['P_out_grid_total'].mean()
    P_pass_nostor[x,y,:] = datanow['P_out_pass_total'].mean()
    P_inv_nostor[x,y,:] = datanow['P_inv_in_total'].mean() * eta_inv

for x, y in [i for i in it.product(range(len(regions_list)),
                                   range(len(P_pc_out_maxs_range)))]:
    
    oversize = 0
    
    datanow = select_data(data, years, regions[x,0,y], 
                          P_pc_out_max = P_pc_out_maxs[x,0,y],
                          E_st_max = E_st_maxs[x,0,y],
                          P_st_in_max = P_st_in_maxs[x,0,y],
                          oversize = oversize)
    
#    print(datanow['C_out_grid_total'])
#    data, year, region=None, oversize=None, eta_st=None, 
#                E_st_max=None, P_st_in_max=None, P_pc_out_max=None, eta_inv=None
    
    C_pur_nopv[x,:,y] = (datanow['C_in_grid_total'] * exc_rate.loc[years]['AUD/USD']).mean()
    C_gen_nopv[x,:,y] = (datanow['C_out_grid_total'] * exc_rate.loc[years]['AUD/USD']).mean()
    P_pur_nopv[x,:,y] = (datanow['P_in_grid_total']).mean() 
    P_gen_nopv[x,:,y] = (datanow['P_out_grid_total']).mean()
    
    
C_st_out = C_gen - C_inv
C_st_tot = C_pur + C_cur + C_vol
C_st_val = C_st_out / C_st_tot
P_st_out = P_gen - P_inv

C_st_out_nopv = C_gen_nopv
C_st_tot_nopv = C_pur_nopv
C_st_val_nopv = C_st_out_nopv / C_st_tot_nopv
P_st_out_nopv = P_gen_nopv

revenue = C_gen - C_pur
revenue_nostor = C_gen_nostor - C_pur_nostor
revenue_nopv = C_gen_nopv - C_pur_nopv

CAPEX_ravel = np.ravel(CAPEX)
CAPEX_nostor_ravel = np.ravel(CAPEX_nostor)
CAPEX_nopv_ravel = np.ravel(CAPEX_nopv)
OM_ravel = np.ravel(OM)
rev_ravel = np.ravel(revenue)
rev_nostor_ravel = np.ravel(revenue_nostor)
rev_nopv_ravel = np.ravel(revenue_nopv)
#cashflows = [-CAPEX] + [revenue - OM] * lifetime

#IRRs = np.array([np.irr([-CAPEX] + [rev_ravel[i] - OM_ravel[i]] * lifetime) for i in range(len(rev_ravel))]).reshape(OM.shape)

IRRs = np.array([np.irr([-CAPEX_ravel[i]] + [rev_ravel[i] - OM_ravel[i]] * lifetime) \
                 for i in range(len(rev_ravel))]).reshape(OM.shape)

IRRs_nostor = np.array([np.irr([-CAPEX_nostor_ravel[i]] + [rev_nostor_ravel[i] - OM_ravel[i]] * lifetime) \
                        for i in range(len(rev_nostor_ravel))]).reshape(OM.shape)

IRRs_nopv = np.array([np.irr([-CAPEX_nopv_ravel[i]] + [rev_nopv_ravel[i]] * lifetime) \
                        for i in range(len(rev_nostor_ravel))]).reshape(OM.shape)

IRRs_nostor_diff = IRRs - IRRs_nostor 

IRRs_nopv_diff = IRRs - IRRs_nopv

plot_data = [IRRs*100, IRRs_nostor*100, IRRs_nostor_diff*100, IRRs_nopv*100, IRRs_nopv_diff*100]
               
for i in range(len(plot_data)):
               
    ax1[i].plot_surface(oversizes[0], P_pc_out_maxs[0], plot_data[i][0], cmap = 'inferno', alpha = 0.5)
    ax1[i].plot_surface(oversizes[1], P_pc_out_maxs[1], plot_data[i][1], cmap = 'viridis', alpha = 0.5)
    ax1[i].set_zlim([np.amin(plot_data[i]) * (1.0 - 0.1*np.sign(np.amin(plot_data[i]))), 1.1*np.amax(plot_data[i])])

    
    ax8[2*i].contourf(oversizes[0], P_pc_out_maxs[0], plot_data[i][0], 50, cmap = 'inferno')
    ax8[2*i+1].contourf(oversizes[1], P_pc_out_maxs[1], plot_data[i][1], 50, cmap = 'viridis')
    cont1 = ax8[2*i].contour(oversizes[0], P_pc_out_maxs[0], plot_data[i][0], 
                     levels = np.linspace(np.amin(plot_data[i][0]), 
                                          np.amax(plot_data[i][0]), 10), colors = 'k')
    cont2 = ax8[2*i+1].contour(oversizes[1], P_pc_out_maxs[1], plot_data[i][1], 
                       levels = np.linspace(np.amin(plot_data[i][1]), 
                                            np.amax(plot_data[i][1]), 10), colors = 'k')
    
    ax8[2*i].clabel(cont1, inline_spacing=5, colors = 'k', fmt = '$%2.2f$', fontsize=14)
    ax8[2*i+1].clabel(cont2, inline_spacing=5, colors = 'k', fmt = '$%2.2f$', fontsize=14)
               

for i in P_pc_out_maxs_range:

    bar_idx_NSW = (P_pc_out_maxs == i) & (regions == 'NSW')
    bar_idx_SA = (P_pc_out_maxs == i) & (regions == 'SA')
    
    alpha = 0.5 + (np.amax(P_pc_out_maxs_range) - i)/np.amax(P_pc_out_maxs_range)/2
    
    ax21.bar3d(oversizes[bar_idx_NSW], i, z = 0, dx = 0.05, dy = 5, 
               dz = C_pur[bar_idx_NSW], label = 'purchased', 
               color = 'b', alpha = alpha, shade = True)

    ax21.bar3d(oversizes[bar_idx_NSW], i, z = C_pur[bar_idx_NSW], dx = 0.05, dy = 5, 
               dz = C_cur[bar_idx_NSW], label = 'curtailed', 
               color = 'orange', alpha = alpha, shade = True)

    ax21.bar3d(oversizes[bar_idx_NSW], i, z = C_pur[bar_idx_NSW] + C_cur[bar_idx_NSW], 
               dx = 0.05, dy = 5, dz = C_vol[bar_idx_NSW], label = 'voluntary', 
               color = 'g', alpha = alpha, shade = True)

    ax22.bar3d(oversizes[bar_idx_SA], i, z = 0, dx = 0.05, dy = 5, 
               dz = C_pur[bar_idx_SA], label = 'purchased', 
               color = 'b', alpha = alpha, shade = True)

    ax22.bar3d(oversizes[bar_idx_SA], i, z = C_pur[bar_idx_SA], dx = 0.05, dy = 5, 
               dz = C_cur[bar_idx_SA], label = 'curtailed', 
               color = 'orange', alpha = alpha, shade = True)

    ax22.bar3d(oversizes[bar_idx_SA], i, z = C_pur[bar_idx_SA] + C_cur[bar_idx_SA], 
               dx = 0.05, dy = 5, dz = C_vol[bar_idx_SA], label = 'voluntary', 
               color = 'g', alpha = alpha, shade = True)

    ax31.bar3d(oversizes[bar_idx_NSW], i, z = 0, dx = 0.05, dy = 5, 
               dz = C_inv[bar_idx_NSW], label = 'PV out', color = 'b', 
               alpha = alpha, shade = True)

    ax31.bar3d(oversizes[bar_idx_NSW], i, z = C_inv[bar_idx_NSW], dx = 0.05, dy = 5, 
               dz = C_st_out[bar_idx_NSW], label = 'Storage out', color = 'orange', 
               alpha = alpha, shade = True)

    ax32.bar3d(oversizes[bar_idx_SA], i, z = 0, dx = 0.05, dy = 5, 
               dz = C_inv[bar_idx_SA], label = 'PV out', color = 'b', 
               alpha = alpha, shade = True)

    ax32.bar3d(oversizes[bar_idx_SA], i, z = C_inv[bar_idx_SA], dx = 0.05, dy = 5, 
               dz = C_st_out[bar_idx_SA], label = 'Storage out', color = 'orange', 
               alpha = alpha, shade = True)

    ax41.bar3d(oversizes[bar_idx_NSW], i, z = 0, dx = 0.05, dy = 5, 
               dz = P_pur[bar_idx_NSW], label = 'purchased', 
               color = 'b', alpha = alpha, shade = True)

    ax41.bar3d(oversizes[bar_idx_NSW], i, z = P_pur[bar_idx_NSW], dx = 0.05, dy = 5, 
               dz = P_cur[bar_idx_NSW], label = 'curtailed', 
               color = 'orange', alpha = alpha, shade = True)

    ax41.bar3d(oversizes[bar_idx_NSW], i, z = P_pur[bar_idx_NSW] + P_cur[bar_idx_NSW], 
               dx = 0.05, dy = 5, dz = P_vol[bar_idx_NSW], label = 'voluntary', 
               color = 'g', alpha = alpha, shade = True)

    ax42.bar3d(oversizes[bar_idx_SA], i, z = 0, dx = 0.05, dy = 5, 
               dz = P_pur[bar_idx_SA], label = 'purchased', 
               color = 'b', alpha = alpha, shade = True)

    ax42.bar3d(oversizes[bar_idx_SA], i, z = P_pur[bar_idx_SA], dx = 0.05, dy = 5, 
               dz = P_cur[bar_idx_SA], label = 'curtailed', 
               color = 'orange', alpha = alpha, shade = True)

    ax42.bar3d(oversizes[bar_idx_SA], i, z = P_pur[bar_idx_SA] + P_cur[bar_idx_SA], 
               dx = 0.05, dy = 5, dz = P_vol[bar_idx_SA], label = 'voluntary', 
               color = 'g', alpha = alpha, shade = True)
    
    ax51.bar3d(oversizes[bar_idx_NSW], i, z = 0, dx = 0.05, dy = 5, 
               dz = P_inv[bar_idx_NSW], label = 'PV_out', 
               color = 'b', alpha = alpha, shade = True)
        
    ax51.bar3d(oversizes[bar_idx_NSW], i, z = P_inv[bar_idx_NSW], dx = 0.05, dy = 5, 
               dz = P_st_out[bar_idx_NSW], label = 'Storage out', 
               color = 'orange', alpha = alpha, shade = True)
        
    ax52.bar3d(oversizes[bar_idx_SA], i, z = 0, dx = 0.05, dy = 5, 
               dz = P_inv[bar_idx_SA], label = 'PV_out', 
               color = 'b', alpha = alpha, shade = True)
        
    ax52.bar3d(oversizes[bar_idx_SA], i, z = P_inv[bar_idx_SA], dx = 0.05, dy = 5, 
               dz = P_st_out[bar_idx_SA], label = 'Storage out', 
               color = 'orange', alpha = alpha, shade = True)

ax61.plot_surface(oversizes[0], P_pc_out_maxs[0], C_st_val[0], 
                  label = 'Storage out', cmap = 'inferno')

ax62.plot_surface(oversizes[1], P_pc_out_maxs[1], C_st_val[1], 
                  label = 'Storage out', cmap = 'inferno')

ax71.plot_surface(oversizes[0], P_pc_out_maxs[0], C_inv[0]/P_inv[0], cmap = 'inferno')
ax71.plot_surface(oversizes[0], P_pc_out_maxs[0], C_st_out[0]/P_st_out[0], cmap = 'viridis', alpha = 0.5)

ax72.plot_surface(oversizes[1], P_pc_out_maxs[1], C_inv[1]/P_inv[1], cmap = 'inferno')
ax72.plot_surface(oversizes[1], P_pc_out_maxs[1], C_st_out[1]/P_st_out[1], cmap = 'viridis', alpha = 0.5)

ax71.set_zlim([0,1.1*np.amax(C_st_out[0]/P_st_out[0])])
ax72.set_zlim([0,1.1*np.amax(C_st_out[1]/P_st_out[1])])

#ax72.plot_surface(oversizes[0], P_st_in_maxs[0], C_inv[0]/P_inv[0], cmap = 'inferno', label = 'PV out')
#ax72.plot_surface(oversizes[1], P_st_in_maxs[1], C_inv[1]/P_inv[1], cmap = 'viridis', label = 'Storage out')
#        ax72.bar(oversize-0.02, C_inv[x,y]/P_inv[x,y], width = 0.04, color = 'b', label = 'PV out', zorder = 2)
#        ax72.bar(oversize+0.02, C_st_out[x,y]/P_st_out[x,y], width = 0.04, color = 'orange', label = 'storage out', zorder = 2)
            
ax11.view_init(30, 45)
ax12.view_init(30, 45)
ax13.view_init(30, 45)
ax14.view_init(30, 45)
ax15.view_init(30, 45)
    
#ax21.view_init(30, -115)
#ax22.view_init(30, -115)
#ax31.view_init(30, -115)
#ax32.view_init(30, -115)
#ax41.view_init(30, -115)
#ax42.view_init(30, -115)
#ax51.view_init(30, -115)
#ax52.view_init(30, -115)
#ax61.view_init(30, -115)
#ax62.view_init(30, -115)

#ax2_handles, ax2_labels = ax21.get_legend_handles_labels()
#ax2_label = dict(zip(ax2_labels, ax2_handles))

#ax3_handles, ax3_labels = ax31.get_legend_handles_labels()
#ax3_label = dict(zip(ax3_labels, ax3_handles))

#ax4_handles, ax4_labels = ax41.get_legend_handles_labels()
#ax4_label = dict(zip(ax4_labels, ax4_handles))

#ax5_handles, ax5_labels = ax51.get_legend_handles_labels()
#ax5_label = dict(zip(ax5_labels, ax5_handles))

#ax7_handles, ax7_labels = ax71.get_legend_handles_labels()
#ax7_label = dict(zip(ax7_labels, ax7_handles))

#ax21.legend(ax2_label.values(), ax2_label.keys())
#ax21.set_ylim([0,5e6])
#ax22.legend(ax2_label.values(), ax2_label.keys())
#ax22.set_ylim([0,5e6])

#ax31.legend(ax3_label.values(), ax3_label.keys())
#ax31.set_ylim([0,2.5e7])
#ax32.legend(ax3_label.values(), ax3_label.keys())
#ax32.set_ylim([0,2.5e7])

#ax41.legend(ax4_label.values(), ax4_label.keys())
#ax41.set_ylim([0,1e5])
#ax42.legend(ax4_label.values(), ax4_label.keys())
#ax42.set_ylim([0,1e5])

#ax51.legend(ax5_label.values(), ax5_label.keys())
#ax51.set_ylim([0,2e5])
#ax52.legend(ax5_label.values(), ax5_label.keys())
#ax52.set_ylim([0,2e5])

#ax61.set_ylim([0,7])

#ax62.set_ylim([0,7])
#ax62.grid(True)

#ax71.legend(ax7_label.values(), ax7_label.keys())
#ax71.set_ylim([0,400])
#ax72.legend(ax7_label.values(), ax7_label.keys())
#ax72.set_ylim([0,400])

#savedir = basedir + 'Publications/paper_2/Figures/'

## Find the optimal charging and discharging rate 

In [ ]:
from numpy import arange, empty, average, meshgrid, irr, array, around
from package import sql_manager as sm
import itertools as it
from scipy.interpolate import LinearNDInterpolator

data = sm.get_table('storage_value_PF_PVplant_normalised2.db', 'summary')


regionlist = ['SA', 'NSW']
yearlist = arange(2016, 2020)
SPHlist = arange(0.05, 1.01, 0.05)
CPHlist = arange(0.05, 1.01, 0.05)
etalist = arange(0.4, 0.91, 0.1)

revenue = empty((len(etalist), len(regionlist), len(yearlist),
                          len(SPHlist), len(CPHlist)))

C_in_grid_total = empty((len(etalist), len(regionlist), len(yearlist),
                          len(SPHlist), len(CPHlist)))

C_out_grid_total = empty((len(etalist), len(regionlist), len(yearlist),
                          len(SPHlist), len(CPHlist)))

points = [i for i in it.product(SPHlist, CPHlist)]
indices = [i for i in it.product(range(len(etalist)), range(len(regionlist)), range(len(yearlist)), 
                                 range(len(SPHlist)), range(len(CPHlist)))]
    
for e, r, y, s, c in indices:
    
    datanow = data.loc[regionlist[r]]
    
    etabools = around(array(datanow['eta_pc']).astype(float), 1) == around(etalist[e], 1)
    yearbools = datanow['year']==yearlist[y]
    SPHbools = around(array(datanow['SPH']).astype(float), 2) == around(SPHlist[s], 2)
    CPHbools = around(array(datanow['CPH']).astype(float), 2) == around(CPHlist[c], 2)
    
    datanow = datanow[etabools & yearbools & SPHbools & CPHbools]
    
    C_in_grid_total[e,r,y,s,c] = datanow['C_in_grid_total']
    C_out_grid_total[e,r,y,s,c] = datanow['C_out_grid_total']
    
#==== set up the optimisation approach ====

#SA_revavg_interp = LinearNDInterpolator(points, revenue_avg[0].ravel())
#NSW_revavg_interp = LinearNDInterpolator(points, revenue_avg[1].ravel())

In [ ]:
from numpy import arange, empty, average, meshgrid, irr, array, where, linspace, amin
import itertools as it
import matplotlib.pyplot as mpl

USD2AUD = 1.35

E_st_max = 200
lifetime = 20

eta_pc = 0.4
e = 0
etalist = arange(0.4, 0.91, 0.1)
e = where(etalist == 0.4)[0]

#CH = axis 0
#SH = axis 1

SPH, CPH = meshgrid(arange(0.05, 1.01, 0.05), arange(0.05, 1.01, 0.05))

P_pc_out_max = E_st_max * SPH
P_st_in_max = E_st_max * CPH / eta_pc
Q_st_max = E_st_max / eta_pc

CPC = 30e3 * USD2AUD
CPP = 12e3 * USD2AUD    #Cost per MW-e
CPE = 25e3 * USD2AUD      #Cost per MWh-th

#CPC = 208.33e3 / 2 * USD2AUD    #Cost per MW-e
#CPP = 208.33e3 / 2 * USD2AUD
#CPE = 66.67e3 * USD2AUD

C_in_grid_total_scaled = C_in_grid_total / eta_pc * E_st_max
C_out_grid_total_scaled = C_out_grid_total * E_st_max

revenue = C_out_grid_total_scaled - C_in_grid_total_scaled
revenue_avg = average(revenue, axis = 2)
revenue_avg_SA = revenue_avg[e,0][0]
revenue_avg_NSW = revenue_avg[e,1][0]

C_st = Q_st_max * CPE
C_pc = P_pc_out_max * CPP
C_ch = P_st_in_max * CPC

CAPEX = C_st + C_pc + C_ch

irr_indices = array([i for i in it.product(range(len(SPHlist)), range(len(CPHlist)))])
irr_points = array([i for i in it.product(SPHlist, CPHlist)])

IRR_lists_SA = array([[-CAPEX[i[0],i[1]]]+[revenue_avg_SA[i[0],i[1]]]*lifetime \
                       for i in irr_indices])

IRR_lists_NSW = array([[-CAPEX[i[0],i[1]]]+[revenue_avg_SA[i[0],i[1]]]*lifetime \
                       for i in irr_indices])

IRRs_SA = array([irr([-CAPEX[i[0],i[1]]]+[revenue_avg_SA[i[0],i[1]]]*lifetime) \
                 for i in irr_indices]).reshape(CAPEX.shape)

IRRs_NSW = array([irr([-CAPEX[i[0],i[1]]]+[revenue_avg_NSW[i[0],i[1]]]*lifetime) \
                 for i in irr_indices]).reshape(CAPEX.shape)

CPH_test = 0.25
SPH_test = 0.25

fig = mpl.figure(figsize = (260 / 25.4, 100 / 25.4))
ax11 = fig.add_subplot(121)
cont = ax11.contourf(SPH, CPH, IRRs_SA, 20, cmap = 'inferno')
ax11.plot(SPH_test, CPH_test, 'x')
fig.colorbar(cont)
ax11.set_xlabel('Discharge per Hour')
ax11.set_ylabel('Charge per Hour')
ax11.set_title('SA')
ax11.axis([0,1,0,1])
ax11.grid(True)
ax11.grid(True, linestyle = ':')

ax12 = fig.add_subplot(122)
cont = ax12.contourf(SPH, CPH, IRRs_NSW, 20, cmap = 'inferno')
fig.colorbar(cont)
ax12.set_xlabel('Discharge per Hour')
ax12.set_ylabel('Charge per Hour')
ax12.set_title('NSW')
ax12.axis([0,1,0,1])
ax12.grid(True, linestyle = ':')

#=== test for comparison to previous sims ===

test_idx = int((CPH_test-0.05) * 400 + SPH_test * 20 - 1)

IRR_list_test = array(IRR_lists_SA[test_idx])
IRR_test = irr(IRR_lists_SA[test_idx])

print(IRR_list_test)
print(IRR_test)
print(CAPEX.ravel()[test_idx])
print(P_pc_out_max.ravel()[test_idx])
print(P_st_in_max.ravel()[test_idx])
print(C_in_grid_total_scaled.ravel()[test_idx])
print(C_out_grid_total_scaled.ravel()[test_idx])
print(revenue_avg.ravel()[test_idx])